In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [2]:
digits = load_digits()
digits_y = np.eye(10)[digits.target]
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits_y,
                                                   test_size=0.2, random_state=1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1437, 64), (1437, 10), (360, 64), (360, 10))

In [9]:
y_test[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

# 低阶API构建

In [10]:
class Model(object):
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal([64,30]))
        self.b1 = tf.Variable(tf.random.normal([30]))
        self.W2 = tf.Variable(tf.random.normal([30,10]))
        self.b2 = tf.Variable(tf.random.normal([10]))
    
    def __call__(self, x):
        x = tf.cast(x, tf.float32)
        
        fc1 = tf.nn.relu(tf.add(tf.matmul(x, self.W1), self.b1))
        fc2 = tf.add(tf.matmul(fc1, self.W2), self.b2)
        return fc2

In [11]:
def loss_fn(model, x, y):
    preds = model(x)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y))

In [12]:
def accuracy_fn(logits, labels):
    preds = tf.argmax(logits, axis=1)
    labels = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

In [15]:
EPOCHS = 100
LEARNING_RATE = 0.02
model = Model()

for epoch in range(EPOCHS):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, X_train, y_train)
    
    trainable_variables = [model.W1, model.b1, model.b1, model.W2, model.b2]
    grads = tape.gradient(loss, trainable_variables)
    
    optimizer = tf.optimizers.Adam(learning_rate=LEARNING_RATE)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    accuracy = accuracy_fn(model(X_test),y_test)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{EPOCHS}], Train loss: {loss}, Test accuracy: {accuracy}')

Epoch [10/100], Train loss: 62.54087829589844, Test accuracy: 1.0
Epoch [20/100], Train loss: 25.038129806518555, Test accuracy: 1.0
Epoch [30/100], Train loss: 17.026206970214844, Test accuracy: 1.0
Epoch [40/100], Train loss: 14.189234733581543, Test accuracy: 1.0
Epoch [50/100], Train loss: 12.31187915802002, Test accuracy: 1.0
Epoch [60/100], Train loss: 10.96162223815918, Test accuracy: 1.0
Epoch [70/100], Train loss: 10.112123489379883, Test accuracy: 1.0
Epoch [80/100], Train loss: 9.474594116210938, Test accuracy: 1.0
Epoch [90/100], Train loss: 9.049287796020508, Test accuracy: 1.0
Epoch [100/100], Train loss: 8.688253402709961, Test accuracy: 1.0


# Keras高阶API实现

##### 顺序模型

In [17]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=30,input_dim=64,activation='relu'))
model.add(tf.keras.layers.Dense(units=10,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                1950      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                310       
Total params: 2,260
Trainable params: 2,260
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_test,y_test))

Train on 1437 samples, validate on 360 samples
Epoch 1/20
1437/1437 [==============================] - 1s 483us/sample - loss: 5.3006 - accuracy: 0.0877 - val_loss: 3.2257 - val_accuracy: 0.2056
Epoch 2/20
1437/1437 [==============================] - 0s 31us/sample - loss: 2.6535 - accuracy: 0.2714 - val_loss: 1.9372 - val_accuracy: 0.4194
Epoch 3/20
1437/1437 [==============================] - 0s 32us/sample - loss: 1.6865 - accuracy: 0.4600 - val_loss: 1.2828 - val_accuracy: 0.5556
Epoch 4/20
1437/1437 [==============================] - 0s 28us/sample - loss: 1.1497 - accuracy: 0.6228 - val_loss: 0.8339 - val_accuracy: 0.7361
Epoch 5/20
1437/1437 [==============================] - 0s 37us/sample - loss: 0.8337 - accuracy: 0.7279 - val_loss: 0.6371 - val_accuracy: 0.7917
Epoch 6/20
1437/1437 [==============================] - 0s 34us/sample - loss: 0.6541 - accuracy: 0.7850 - val_loss: 0.5137 - val_accuracy: 0.8500
Epoch 7/20
1437/1437 [==============================] - 0s 40us/sample

##### 函数式模型

In [20]:

inputs = tf.keras.Input(shape=(64,))
x = tf.keras.layers.Dense(30,activation='relu')(inputs)
outputs = tf.keras.layers.Dense(10,activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                1950      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                310       
Total params: 2,260
Trainable params: 2,260
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_test,y_test))

Train on 1437 samples, validate on 360 samples
Epoch 1/20
1437/1437 [==============================] - 0s 273us/sample - loss: 6.8276 - accuracy: 0.1009 - val_loss: 3.8485 - val_accuracy: 0.1944
Epoch 2/20
1437/1437 [==============================] - 0s 27us/sample - loss: 2.9547 - accuracy: 0.2345 - val_loss: 2.2692 - val_accuracy: 0.3444
Epoch 3/20
1437/1437 [==============================] - 0s 29us/sample - loss: 1.9326 - accuracy: 0.4099 - val_loss: 1.6642 - val_accuracy: 0.4917
Epoch 4/20
1437/1437 [==============================] - 0s 28us/sample - loss: 1.4483 - accuracy: 0.5324 - val_loss: 1.2909 - val_accuracy: 0.5833
Epoch 5/20
1437/1437 [==============================] - 0s 27us/sample - loss: 1.1338 - accuracy: 0.6256 - val_loss: 1.0375 - val_accuracy: 0.6722
Epoch 6/20
1437/1437 [==============================] - 0s 27us/sample - loss: 0.9156 - accuracy: 0.7035 - val_loss: 0.8567 - val_accuracy: 0.7444
Epoch 7/20
1437/1437 [==============================] - 0s 28us/sample

##### 混合模型

In [22]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.dense_1 = tf.keras.layers.Dense(30, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(10, activation='softmax')
    
    def call(self,inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [23]:
model = Model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Train on 1437 samples, validate on 360 samples
Epoch 1/10
1437/1437 [==============================] - 0s 265us/sample - loss: 5.8186 - accuracy: 0.1823 - val_loss: 3.1257 - val_accuracy: 0.2333
Epoch 2/10
1437/1437 [==============================] - 0s 27us/sample - loss: 2.5164 - accuracy: 0.3145 - val_loss: 1.7864 - val_accuracy: 0.4361
Epoch 3/10
1437/1437 [==============================] - 0s 28us/sample - loss: 1.5599 - accuracy: 0.5226 - val_loss: 1.2321 - val_accuracy: 0.5972
Epoch 4/10
1437/1437 [==============================] - 0s 28us/sample - loss: 1.1094 - accuracy: 0.6444 - val_loss: 0.9435 - val_accuracy: 0.6889
Epoch 5/10
1437/1437 [==============================] - 0s 27us/sample - loss: 0.8529 - accuracy: 0.7321 - val_loss: 0.7376 - val_accuracy: 0.7667
Epoch 6/10
1437/1437 [==============================] - 0s 29us/sample - loss: 0.6855 - accuracy: 0.7850 - val_loss: 0.6161 - val_accuracy: 0.8056
Epoch 7/10
1437/1437 [==============================] - 0s 27us/sample